In [ ]:
import pickle
import numpy as np

# ============================================================
# 0) MASTER SANITY CHECK (la función)
# ============================================================
def sanity_check_tortuous_vs_nontortuous(
    edges,
    r_edge,
    edge_length,
    points_by_edge,
    r_point,
    lengths2_by_edge,
    tol=1e-6,
):
    print("\n==============================")
    print(" SANITY CHECK TORTUOUS VS NON ")
    print("==============================\n")

    ratios_length = []
    diff_r_mean = []
    diff_r_median = []
    bad_edges = []

    # Para global stats
    D_edge = []
    D_tort_mean = []
    L_edge_all = []
    L2_all = []

    skipped_no_pts = 0
    skipped_missing = 0

    for e in edges:
        if e not in points_by_edge or e not in lengths2_by_edge or e not in r_edge or e not in edge_length:
            skipped_missing += 1
            continue

        pts = points_by_edge[e]
        if pts is None or len(pts) < 2:
            skipped_no_pts += 1
            continue

        r_pts = np.array([r_point[p] for p in pts], dtype=float)
        r_e = float(r_edge[e])

        r_mean = float(np.mean(r_pts))
        r_median = float(np.median(r_pts))

        diff_r_mean.append(r_mean - r_e)
        diff_r_median.append(r_median - r_e)

        L_edge = float(edge_length[e])
        L2 = float(np.sum(lengths2_by_edge[e]))

        if L2 + tol < L_edge:
            bad_edges.append(e)

        ratios_length.append(L2 / L_edge if L_edge > 0 else np.nan)

        D_edge.append(2.0 * r_e)
        D_tort_mean.append(2.0 * r_mean)
        L_edge_all.append(L_edge)
        L2_all.append(L2)

    D_edge = np.array(D_edge, dtype=float)
    D_tort_mean = np.array(D_tort_mean, dtype=float)
    L_edge_all = np.array(L_edge_all, dtype=float)
    L2_all = np.array(L2_all, dtype=float)
    ratios_length = np.array(ratios_length, dtype=float)

    def safe_mean(x):
        x = np.asarray(x, dtype=float)
        x = x[np.isfinite(x)]
        return float(np.mean(x)) if len(x) else np.nan

    def safe_std(x):
        x = np.asarray(x, dtype=float)
        x = x[np.isfinite(x)]
        return float(np.std(x)) if len(x) else np.nan

    D_edge_lenw = (np.sum(D_edge * L_edge_all) / np.sum(L_edge_all)) if np.sum(L_edge_all) > 0 else np.nan
    D_tort_lenw = (np.sum(D_tort_mean * L2_all) / np.sum(L2_all)) if np.sum(L2_all) > 0 else np.nan

    print("0️⃣ COUNTS")
    print(f"Edges provided              : {len(edges)}")
    print(f"Edges skipped (missing data): {skipped_missing}")
    print(f"Edges skipped (no pts)      : {skipped_no_pts}")
    print(f"Edges analyzed              : {len(D_edge)}\n")

    print("1️⃣ RADIUS CONSISTENCY (edge vs point)")
    print(f"Mean(r_pts) - r_edge   : mean={safe_mean(diff_r_mean):.6f}  std={safe_std(diff_r_mean):.6f}")
    print(f"Median(r_pts) - r_edge : mean={safe_mean(diff_r_median):.6f}  std={safe_std(diff_r_median):.6f}\n")

    print("2️⃣ LENGTH CONSISTENCY")
    print(f"mean(L2/L_edge) = {safe_mean(ratios_length):.6f}")
    print(f"min(L2/L_edge)  = {np.nanmin(ratios_length):.6f}")
    print(f"max(L2/L_edge)  = {np.nanmax(ratios_length):.6f}")
    print(f"Edges with L2 < L_edge : {len(bad_edges)}\n")

    print("3️⃣ DIAMETER COMPARISON (unweighted)")
    print(f"Non-tortuous (edge mean)        : {safe_mean(D_edge):.6f} µm")
    print(f"Tortuous (mean of per-edge mean): {safe_mean(D_tort_mean):.6f} µm\n")

    print("4️⃣ LENGTH-WEIGHTED DIAMETER")
    print(f"Non-tortuous (length-weighted)  : {D_edge_lenw:.6f} µm")
    print(f"Tortuous (length2-weighted)     : {D_tort_lenw:.6f} µm\n")

    print("🧠 INTERPRETATION HINTS")
    if abs(safe_mean(diff_r_mean)) > 1e-3:
        print("⚠️ r_edge ≠ mean(r_pts) (bias sistemático) → DEFINICIÓN distinta o mapeo edge↔polyline mal.")
    if len(bad_edges) > 0:
        print("❌ Hay edges con L2 < L_edge → BUG geométrico / lengths2 mal construidas.")
    if np.isfinite(D_edge_lenw) and np.isfinite(D_tort_lenw) and abs(D_edge_lenw - D_tort_lenw) > 1e-3:
        print("⚠️ Cambia al ponderar por longitud → EFECTO de weighting/tortuosidad (no necesariamente calibre).")

    print("\n==============================\n")

    return {
        "diff_r_mean": diff_r_mean,
        "diff_r_median": diff_r_median,
        "length_ratios": ratios_length,
        "bad_edges": bad_edges,
        "D_edge_lenw": D_edge_lenw,
        "D_tort_lenw": D_tort_lenw,
    }


# ============================================================
# 1) Helpers: carga + inspección + selección automática attrs
# ============================================================
def load_pkl(path):
    with open(path, "rb") as f:
        return pickle.load(f)

def print_data_summary(name, data):
    print(f"\n================ {name} SUMMARY ================")
    print("Top-level keys:", sorted(list(data.keys())))

    G = data.get("graph", None)
    if G is None:
        print("❌ data['graph'] no existe")
        return

    print(f"Graph: v={G.vcount()}  e={G.ecount()}")
    print("Vertex attributes:", G.vs.attributes())
    print("Edge attributes  :", G.es.attributes())

def pick_first_existing_attr(attr_list, candidates):
    for c in candidates:
        if c in attr_list:
            return c
    return None

def build_edge_map_from_attr(G, attr_name):
    """Devuelve dict {edge_id: value} para un atributo de edge."""
    out = {}
    for e in G.es:
        out[e.index] = e[attr_name]
    return out

def ensure_edge_ids_consistent(keys_a, keys_b, label_a="A", label_b="B"):
    sa, sb = set(keys_a), set(keys_b)
    inter = sa & sb
    only_a = sa - sb
    only_b = sb - sa
    print(f"\n--- EdgeID consistency {label_a} vs {label_b} ---")
    print(f"Common edges: {len(inter)}")
    print(f"Only {label_a}: {len(only_a)}")
    print(f"Only {label_b}: {len(only_b)}")
    return inter, only_a, only_b


# ============================================================
# 2) CONFIG: pon aquí tus dos archivos
# ============================================================
p_non  = "/home/admin/Ana/MicroBrain/output/graph_18_OutGeom.pkl"
p_tort = "/home/admin/Ana/MicroBrain/18_igraph.pkl"  # <-- cambia al nombre real


# ============================================================
# 3) LOAD + INSPECT
# ============================================================
data_non  = load_pkl(p_non)
data_tort = load_pkl(p_tort)

print_data_summary("NON-TORTUOUS", data_non)
print_data_summary("TORTUOUS", data_tort)


# ============================================================
# 4) Extract graphs
# ============================================================
G_non  = data_non["graph"]
G_tort = data_tort["graph"]

# ============================================================
# 5) Detect edge radius / length attributes en NON (source of truth)
# ============================================================
edge_attrs_non = G_non.es.attributes()

radius_candidates = ["r_edge", "radius_edge", "radius", "r", "rad", "radius_um", "r_um", "diameter", "diam", "diam_um"]
length_candidates = ["length", "len", "length_um", "L", "edge_length", "length_geom", "length_eucl"]

rad_attr = pick_first_existing_attr(edge_attrs_non, radius_candidates)
len_attr = pick_first_existing_attr(edge_attrs_non, length_candidates)

print("\n================ ATTRIBUTE PICKING ================")
print("NON edge attrs:", edge_attrs_non)
print("Picked radius attr:", rad_attr)
print("Picked length attr:", len_attr)

if rad_attr is None:
    raise KeyError("No encuentro atributo de radio en edges del NON. Mira la lista 'Edge attributes' e indica el correcto.")
if len_attr is None:
    raise KeyError("No encuentro atributo de length en edges del NON. Mira la lista 'Edge attributes' e indica el correcto.")

# Si detectó 'diameter' como radio, lo convertimos a radio
is_diameter = rad_attr in ["diameter", "diam", "diam_um"]

r_edge = build_edge_map_from_attr(G_non, rad_attr)
if is_diameter:
    r_edge = {k: float(v) / 2.0 for k, v in r_edge.items()}

edge_length = build_edge_map_from_attr(G_non, len_attr)


# ============================================================
# 6) Detect tortuous structures: points_by_edge, radii_geom, lengths2_by_edge
# ============================================================
print("\n================ TORTUOUS KEYS PICKING ================")
tkeys = data_tort.keys()
print("TORT top keys:", sorted(list(tkeys)))

# candidatos típicos
pbe_candidates = ["points_by_edge", "edge_points", "polyline_points_by_edge", "edge_to_points"]
rpt_candidates = ["radii_geom", "radius_point", "r_point", "radii_point", "radius_p", "r_geom"]
l2_candidates  = ["lengths2_by_edge", "lengths2", "seg_lengths2_by_edge", "lengths_tort_by_edge"]

pbe_key = pick_first_existing_attr(list(tkeys), pbe_candidates)
rpt_key = pick_first_existing_attr(list(tkeys), rpt_candidates)
l2_key  = pick_first_existing_attr(list(tkeys), l2_candidates)

print("Picked points_by_edge key:", pbe_key)
print("Picked r_point key      :", rpt_key)
print("Picked lengths2 key     :", l2_key)

if pbe_key is None:
    raise KeyError("No encuentro points_by_edge en TORT. Mira keys y pon el nombre correcto.")
if rpt_key is None:
    raise KeyError("No encuentro radii por punto (radii_geom) en TORT. Mira keys y pon el nombre correcto.")
if l2_key is None:
    raise KeyError("No encuentro lengths2_by_edge en TORT. Mira keys y pon el nombre correcto.")

points_by_edge = data_tort[pbe_key]
r_point = data_tort[rpt_key]
lengths2_by_edge = data_tort[l2_key]

# Normaliza: por si vienen como listas indexadas por edge_id en vez de dict
def normalize_edge_container(x, name):
    if isinstance(x, dict):
        return x
    if isinstance(x, (list, tuple)):
        # asumimos que x[edge_id] existe
        return {i: x[i] for i in range(len(x))}
    raise TypeError(f"{name} tiene tipo raro: {type(x)}")

points_by_edge = normalize_edge_container(points_by_edge, "points_by_edge")
lengths2_by_edge = normalize_edge_container(lengths2_by_edge, "lengths2_by_edge")

# ============================================================
# 7) Edge ID consistency check
# ============================================================
common_edges, only_non, only_tort = ensure_edge_ids_consistent(
    r_edge.keys(), points_by_edge.keys(), "NON(r_edge)", "TORT(points_by_edge)"
)
common_edges2, _, _ = ensure_edge_ids_consistent(
    edge_length.keys(), lengths2_by_edge.keys(), "NON(edge_length)", "TORT(lengths2_by_edge)"
)

edges = sorted(list(common_edges & common_edges2))
print(f"\n✅ Using {len(edges)} common edges for sanity check.")


# ============================================================
# 8) RUN SANITY CHECK
# ============================================================
out = sanity_check_tortuous_vs_nontortuous(
    edges=edges,
    r_edge=r_edge,
    edge_length=edge_length,
    points_by_edge=points_by_edge,
    r_point=r_point,
    lengths2_by_edge=lengths2_by_edge,
)

# Si quieres ver cuáles son los edges malos:
if len(out["bad_edges"]) > 0:
    print("First 20 bad_edges:", out["bad_edges"][:20])
